Let $G$ be a permutation group of $X$. If $\alpha \in X$, then the set
\begin{equation}
    \mathop{Orb}(\alpha) = \{g\alpha \in X : g \in G\}
\end{equation}
is called the orbit of $\alpha$ under $G$. If $\beta \in X$ is in the orbit of $\alpha$, then an element $g \in G$ is called a witness of this if $g\alpha = \beta$. We can see that $\beta$ is in the orbit of $\alpha$ if and only if the orbit of $\beta$ is the same as the orbit of $\alpha$. Hence different orbits are disjoint form a partition of $X$.

The stabilizer of an element $\alpha$ in $X$ is
\begin{equation}
G_\alpha = \{g \in G : g\alpha = \alpha\},
\end{equation}
and it is a subgroup of $G$.

Consider the set of all left cosets of $G_\alpha$ in $G$
\begin{equation}
    G/G_\alpha = \{ gG_\alpha : g \in G \}.
\end{equation}
Define a function $f: G/G_\alpha \to \mathop{Orb}(\alpha)$ via $f(gG_\alpha) = g\alpha$.

*   The function is well-defined: If $g_1G_\alpha = g_2G_\alpha$ for $g_1, g_2 \in G$, then $g_2^{-1}g_1 \in G_\alpha$. By the definition of the stabilizer $(g_2^{-1}g_1)\alpha = \alpha$. Applying g₂ to both sides, we get $g_1\alpha = g_2\alpha$, hence $f(g_1G_\alpha) = f(g_2G\alpha)$.

*   The function is injective: Suppose that $f(g_1G_\alpha) = f(g_2G_\alpha)$. By definition, this means that $g_1\alpha = g_2\alpha$. Applying $g_2^{-1}$ to both sides gives $(g_2^{-1}g_1)\alpha = \alpha$. So $g_2^{-1}g_1 \in G_\alpha$ and the left coset $(g_2^{-1}g_1)G_\alpha$ is equal to $G_\alpha$. Multiplying by $g_2$ on the left yields $g_1G_\alpha = g_2G_\alpha$.

*   The function is surjective:
Let $\beta \in \mathop{Orb(\alpha)}$. By the definition, there exists some $g \in G$ such that $g\alpha = \beta$. The left coset $gG_\alpha$ is in the domain of $f$, and $f(gG_\alpha) = g\alpha = \beta$, as required.

This establishes a one-to-one correspondence between the set of left cosets of the stabilizer and the elements of the orbit. This leads directly to the orbit-stabilizer theorem.

Let $G$ be a finite group acting on a set $X$, and let $\alpha \in X$. Then
\begin{equation}
    |G| = |\mathop{Orb}(\alpha)| |G_\alpha|.
\end{equation}
This come directly from Lagrange's theorem applied to our bijection of the left cosets of the stabiliser subgroup and the orbit of $\alpha$.

We describe a proceedure which computes the orbit with witnesses of a given element under a permutation group $G$, generated by a given set of permutations. The procedure is a breadth first search on a graph. The elements of the set $X = \{1, 2, \dots, n\}$ are the nodes of the graph, and the generators of the group $G$ represent the edges. Applying a generator $\pi$ to an element $\beta$ takes you from node $\beta$ to node $\pi(\beta)$. The goal is to find all nodes reachable from the starting element $\alpha$.

1.  Initialisation:
    *   We start with a queue and add our initial element $\alpha$ to it.
    *   We use a dictionary map each element $\beta$ found in the orbit to a witness $g$ such that $g(\alpha) = \beta$.
    *   We initialize this dictionary with the witness for $\alpha$ as the identity permutation.

2.  Exploration Loop (BFS):
    *   The algorithm proceeds by taking an element $\beta$ from the front of the queue.
    *   For this $\beta$, we look up its known witness $g_\beta$.
    *   We then apply every generator $\pi$ to $\beta$ to find all "neighboring" elements $\gamma = \pi(\beta)$.
    *   For each new element $\gamma$:
        *   If $\gamma$ is not a key in our witnesses dictionary, then we have discovered a new part of the orbit.
        *   Add $\gamma$ to the witnesses dictionary. Its witness is calculated as the product $\pi g_\beta$. This is because $(\pi g_\beta)(\alpha) = \pi(g_\beta(\alpha)) = \pi(\beta) = \gamma$.
        *   Add the new element $\gamma$ to the back of the queue to ensure its neighbors are explored later.
3.  Termination:
    *   The loop continues until the queue is empty, which means there are no more elements to process and no new reachable elements can be found.
    *   The final witnesses dictionary contains every element in the orbit of $\alpha$ and a corresponding witness permutation. The procedure then returns this result.

In [11]:
import collections

class Permutation:
    '''
    A class to represent a permutation of the set {1, 2, ..., n}.
    '''
    def __init__(self, p_list):
        self._p = list(p_list)
        self.size = len(p_list)

    def __len__(self):
        return self.size

    def __repr__(self):
        return f"Perm({self._p})"

    def __eq__(self, other):
        if not isinstance(other, Permutation):
            return NotImplemented
        return self._p == other._p

    def __getitem__(self, i):
        if not 1 <= i <= self.size:
            raise IndexError(f"Index {i} is out of bounds for size {self.size}.")
        return self._p[i - 1]

    def __mul__(self, other):
        if not isinstance(other, Permutation) or self.size != other.size:
            raise ValueError("Permutations must be of the same size.")
        new_p_list = [self[other[i]] for i in range(1, self.size + 1)]
        return Permutation(new_p_list)

    def inverse(self):
        inv_p_list = [0] * self.size
        for i in range(1, self.size + 1):
            j = self[i]
            inv_p_list[j - 1] = i
        return Permutation(inv_p_list)

In [12]:
def compute_orbit_with_witnesses(generators, alpha):
    '''
    Computes the orbit of an element alpha under a group G generated by
    a given set of permutations. It also finds a witness permutation for
    each element in the orbit.
    Args:
        generators: The set of generators for the group G.
        alpha: The element whose orbit is to be computed (using 1-based indexing).
    Returns:
        A list where each tuple contains an element of the
        orbit and its corresponding witness permutation.
    '''
    if not generators:
        # If there are no generators, the orbit only contains alpha itself.
        n = alpha
        identity = Permutation(list(range(1, n + 1)))
        return [(alpha, identity)]

    n = generators[0].size
    if not 1 <= alpha <= n:
        raise ValueError(f"Element {alpha} is not in the set {{1, ..., {n}}}.")

    identity = Permutation(list(range(1, n + 1)))
    queue = collections.deque([alpha])
    witnesses = {alpha: identity}

    while queue:
        current_element = queue.popleft()
        current_witness = witnesses[current_element]
        for gen in generators:
            new_element = gen[current_element]
            if new_element not in witnesses:
                witnesses[new_element] = gen * current_witness
                queue.append(new_element)
    return list(witnesses.items())

print("--- Example: The Symmetric Group S3 ---")
pi1 = Permutation([2, 1, 3])
pi2 = Permutation([2, 3, 1])
s3_generators = [pi1, pi2]
orbit_of_1 = compute_orbit_with_witnesses(s3_generators, 1)
print(f"Orbit of 1 with witnesses: {orbit_of_1}")

print("\n--- Example: The Dihedral Group D8 ---")
rotation = Permutation([2, 3, 4, 1])
reflection = Permutation([3, 2, 1, 4])
d8_generators = [rotation, reflection]
orbit_of_2 = compute_orbit_with_witnesses(d8_generators, 2)
print(f"Orbit of 2 with witnesses: {orbit_of_2}")

--- Example: The Symmetric Group S3 ---
Orbit of 1 with witnesses: [(1, Perm([1, 2, 3])), (2, Perm([2, 1, 3])), (3, Perm([3, 2, 1]))]

--- Example: The Dihedral Group D8 ---
Orbit of 2 with witnesses: [(2, Perm([1, 2, 3, 4])), (3, Perm([2, 3, 4, 1])), (4, Perm([3, 4, 1, 2])), (1, Perm([2, 1, 4, 3]))]
